<a href="https://colab.research.google.com/github/SalwaChaouachi/ML/blob/main/SlogansGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 📦 Installations nécessaires
!pip install transformers tensorflow --quiet

# 📚 Imports
import pandas as pd
import random
import numpy as np
from collections import defaultdict
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# 📂 Chargement du dataset
df = pd.read_csv('slogans.csv')

# 👉 Vérifier les noms de colonnes du DataFrame
print(df.columns)

# 👉 Remplacer 'slogan' par le nom correct de la colonne
#    contenant les slogans, par exemple 'Slogan' ou 'Text'
slogans = df['Slogan'].dropna().astype(str).tolist() # or df['Text'] or... depending on your csv


# ✅ 1. N-grammes (Bigrammes simples)
def generate_ngram_text(slogans, n=2, length=10):
    model = defaultdict(list)
    for line in slogans:
        tokens = line.split()
        for i in range(len(tokens) - 1):
            model[tokens[i]].append(tokens[i + 1])
    word = random.choice(list(model.keys()))
    result = [word]
    for _ in range(length):
        next_words = model.get(word)
        if not next_words:
            break
        word = random.choice(next_words)
        result.append(word)
    return ' '.join(result)

print("🔸 N-gramme :", generate_ngram_text(slogans))

# ✅ 2. RNN (Recurrent Neural Network)
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(slogans)
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in slogans:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')
xs, labels = input_sequences[:, :-1], input_sequences[:, -1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

model = Sequential()
model.add(Embedding(total_words, 10, input_length=max_sequence_len - 1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(xs, ys, epochs=10, verbose=0)

def generate_rnn_text(seed_text, next_words=5):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)[0]
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

print("🔸 RNN :", generate_rnn_text("Experience"))

# ✅ 3. GPT-2 (Transformer avec fine-tuning si souhaité)
tokenizer_gpt2 = GPT2Tokenizer.from_pretrained("gpt2")
model_gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")

def generate_gpt2(prompt, max_length=30):
    inputs = tokenizer_gpt2(prompt, return_tensors="pt")
    outputs = model_gpt2.generate(inputs['input_ids'], max_length=max_length, do_sample=True, temperature=0.9)
    return tokenizer_gpt2.decode(outputs[0], skip_special_tokens=True)

print("🔸 GPT-2 :", generate_gpt2("Discover the taste of"))



Index(['Company', 'Slogan'], dtype='object')
🔸 N-gramme : done, and more.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


🔸 RNN : Experience your cup world beautiful hair


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


🔸 GPT-2 : Discover the taste of the rich.

What's better than having a great restaurant and getting to taste what you want. The selection is spot on


In [ ]:
!pip install --upgrade openai


In [ ]:
import getpass
import openai

api_key = getpass.getpass("Enter your OpenAI API key: ")
client = openai.OpenAI(api_key=api_key)


In [2]:
import time
import openai

def generate_gpt_slogan(prompt):
    # Wait for a longer time  before making the API call
    time.sleep(5)  # Increased delay to 5 seconds
    chat_completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.8
    )
    return chat_completion.choices[0].message.content.strip()

# Exemple :
prompt = "Propose un slogan original pour une marque de café tunisienne."
slogan = generate_gpt_slogan(prompt)
print("🧠 GPT-3.5 generated slogan:", slogan)

NameError: name 'client' is not defined